## <center>Naive Bayes Assignment</center>

#### Problem Statement:
<code style="background:lightblue;color:black">To Prepare a classification model using Naive Bayes for salary data.</code>

In [1]:
#Importing the necessary libraries for the model
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics 
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Importing the csv file and having a glance at the test data
salary_Test = pd.read_csv('SalaryData_Test.csv')
salary_Test .head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [3]:
#Importing the csv file and having a glance at the  train data
salary_train = pd.read_csv('SalaryData_Train.csv')
salary_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Making necessary EDA for the model

In [4]:
#looking into the datatyes of the train data and to check if any null values are present
salary_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [5]:
#looking into the datatyes of the train data and to check if any null values are present
salary_Test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [6]:
#Checking into rows and columns of the train data
salary_train.shape

(30161, 14)

In [7]:
#Checking into rows and columns of the train data
salary_Test.shape

(15060, 14)

In [8]:
#Label Encoding the categorical data into numerical 
salary_train = salary_train.apply(LabelEncoder().fit_transform)
salary_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,22,5,9,12,4,0,1,4,1,24,0,39,37,0
1,33,4,9,12,2,3,0,4,1,0,0,12,37,0
2,21,2,11,8,0,5,1,4,1,0,0,39,37,0
3,36,2,1,6,2,5,0,2,1,0,0,39,37,0
4,11,2,9,12,2,9,5,2,0,0,0,39,4,0


In [9]:
#Label Encoding the categorical data into numerical 
salary_Test = salary_Test.apply(LabelEncoder().fit_transform)
salary_Test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,8,2,1,6,4,6,3,2,1,0,0,39,37,0
1,21,2,11,8,2,4,0,4,1,0,0,49,37,0
2,11,1,7,11,2,10,0,4,1,0,0,39,37,1
3,27,2,15,9,2,6,0,2,1,87,0,39,37,1
4,17,2,0,5,4,7,1,4,1,0,0,29,37,0


In [10]:
#Defining the dependent and independent for train dataset
X_train= salary_train.drop(['education','relationship','native','maritalstatus','sex','race'],axis=1)
Y_train= salary_train['Salary']

In [11]:
#Defining the dependent and independent for test dataset
X_test= salary_Test .drop(['education','relationship','native','maritalstatus','sex','race'],axis=1)
Y_test= salary_Test ['Salary']

#### Gaussian Naive Bayes

In [12]:
#Building the Gaussian Model
classifier = GB()
classifier.fit(X_train, Y_train)

GaussianNB()

In [13]:
#Predicting the data
y_pred = classifier.predict(X_test)

In [14]:
#Looking into accuracy and confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
ac = accuracy_score(Y_test,y_pred)
cm = confusion_matrix(Y_test, y_pred)

In [15]:
#Accuracy score for the Gaussian model
ac

1.0

In [16]:
#Confusion Matrix
cm

array([[11360,     0],
       [    0,  3700]], dtype=int64)

In [17]:
accuracy_test = np.mean(y_pred== Y_test)
accuracy_test

1.0

#### Multinominal Naive Bayes

In [18]:
#Building the Multinominal model and checking the accuracy on train data
classifier_mb = MB()
classifier_mb.fit(X_train, Y_train)
classifier_mb.score(X_train, Y_train)
classifier_mb.score(X_test, Y_test)
predicted_result = classifier_mb.predict(X_train)
accuracy_train = np.mean(predicted_result == Y_train)
accuracy_train

0.7800802360664434

In [19]:
#Predicting the data and checking the accuracy on test data
test_predict=classifier_mb.predict(X_test)
accuracy_test_1 = np.mean(test_predict== Y_test)
accuracy_test_1

0.7820053120849934

In [20]:
#Tabulating the accuracy of both the model
Table={'model':['Gaussian NB','Multinomial NB'],
  'accuracy':[accuracy_test,accuracy_test_1]
  }
Table=pd.DataFrame(Table)
Table

,model,accuracy
0,Gaussian NB,1.000000
1,Multinomial NB,0.782005


#### Insights Drawn:
>From both the Models its evident that the Gaussian Models accuracy is best then the Multinominal Model for the given Nave bayes Problem.Hence Gaussian Model Predicts better.